In [1]:
from pydantic import BaseModel, Field, ValidationError, validator, field_validator
from typing import Annotated, Any, Dict, List, Optional, Set, Union
from pydantic.functional_validators import AfterValidator
import uuid
from abstractions.goap.entity import Entity, Attribute,RegistryHolder
from pydantic import BaseModel, Field, ValidationError, ValidationInfo
from pydantic.functional_validators import field_validator

In [2]:
# Specific attribute classes inheriting from Attribute
class Open(Attribute):
    value: bool = Field(default=True, description="Indicates if the entity is open or closed")

class Locked(Attribute):
    value: bool = Field(default=False, description="Indicates if the entity is locked or unlocked")

class Color(Attribute):
    value: str = Field(default="white", description="The color of the entity")

class Material(Attribute):
    value: str = Field(default="wood", description="The material of the entity")


# Door entity inheriting from Entity
class Door(Entity):
    open: Open = Field(default_factory=Open, description="Attribute to indicate door's open state")
    color: Color = Field(default_factory=Color, description="Attribute for the door's color")
    material: Material = Field(default_factory=Material, description="Attribute representing the door's material")

# Lock entity inheriting from Entity
class Lock(Entity):
    locked: Locked = Field(default_factory=Locked, description="Lock state, unlocked or locked")

# LockableDoor entity inheriting from Door and including a Lock entity
class LockableDoor(Door):
    lock: Lock = Field(default_factory=Lock, description="Lock entity for the door")
    gang: str = Field(default="gang", description="gang")
        

#alternatively, you can use the following to inherit from both Door and Lock instead of storing a lock Entity
class DoorLockable(Door,Lock):
    pass

# Example usage
print("Example usage with nested entities:")
try:
    lockable_door = LockableDoor(
        name="Main Entrance",
        open=Open(value=True),
        color=Color(value='white'),
        material=Material(value='wood'),
        lock=Lock(locked=Locked(value=True)),
    )
    print(lockable_door)
except ValidationError as e:
    print(f"Validation error: {e}")
print("\nExample usage with multiple inheritance:")
try:
    door_lockable = DoorLockable(
        name="Exit",
        open=Open(value=True),
        color=Color(value='white'),
        material=Material(value='wood'),
        locked=Locked(value=True),
        
    )
    print(door_lockable)
except ValidationError as e:
    print(f"Validation error: {e}")

Example usage with nested entities:
Validation error: 1 validation error for LockableDoor
gang
  Value error, Attributes must be instances of Attribute or Entity, got str for field gang [type=value_error, input_value='gang', input_type=str]
    For further information visit https://errors.pydantic.dev/2.1.2/v/value_error

Example usage with multiple inheritance:
name='Exit' id='f0335182-a342-427f-867c-55f24847d74d' boh='boh' locked=Locked(name='Locked', id='3ec3922f-0cc9-45c4-b4f2-3dd8d79c45bd', value=True) open=Open(name='Open', id='07434bdf-8910-4ca4-ac17-8c15a49e20cd', value=True) color=Color(name='Color', id='d70c7a49-f1db-4999-b3bc-79a2857328c1', value='white') material=Material(name='Material', id='097fcc2e-e2c1-43f9-b072-2a5b5f27082d', value='wood')


In [6]:
a = Entity()

In [7]:
a

Entity(name='Entity', id='f4ebe0a8-32bf-4c43-ad16-1d76d3d226c4', boh='boh')

In [5]:
lockable_door.boh

NameError: name 'lockable_door' is not defined

In [ ]:
lockable_door.all_attributes()

{'open': Open(name='Open', id='af729b9f-e33b-40f1-b67e-14a226078df3', value=True),
 'color': Color(name='Color', id='d8987489-dbe1-46aa-b4e6-826cc2d7a565', value='white'),
 'material': Material(name='Material', id='89f7198f-5c17-420c-828f-0e0998d7effb', value='wood'),
 'locked': Locked(name='Locked', id='3f2b1d1f-0205-4bf7-ad7b-469c2bb1d373', value=True)}

In [ ]:
Attribute.all_instances()

[Open(name='Open', id='af729b9f-e33b-40f1-b67e-14a226078df3', value=True),
 Color(name='Color', id='d8987489-dbe1-46aa-b4e6-826cc2d7a565', value='white'),
 Material(name='Material', id='89f7198f-5c17-420c-828f-0e0998d7effb', value='wood'),
 Locked(name='Locked', id='3f2b1d1f-0205-4bf7-ad7b-469c2bb1d373', value=True),
 Open(name='Open', id='431ec6fd-27f9-447f-a3a3-0ac6d18e60b6', value=True),
 Color(name='Color', id='fc391112-2404-4461-a170-8cf7b19122a6', value='white'),
 Material(name='Material', id='6a9ad5c5-44fa-47c6-9e32-c796d20d6fa4', value='wood'),
 Locked(name='Locked', id='73282734-295b-4a06-86e7-a49b7159beb9', value=True)]

In [ ]:
RegistryHolder.all_instances_by_type(Entity)

[Lock(name='Lock', id='2a15b9d6-c783-4c52-8c8f-abf913a66e68', locked=Locked(name='Locked', id='3f2b1d1f-0205-4bf7-ad7b-469c2bb1d373', value=True)),
 LockableDoor(name='Main Entrance', id='e6af401c-0bfa-485b-a22a-7b2aa7cfb347', open=Open(name='Open', id='af729b9f-e33b-40f1-b67e-14a226078df3', value=True), color=Color(name='Color', id='d8987489-dbe1-46aa-b4e6-826cc2d7a565', value='white'), material=Material(name='Material', id='89f7198f-5c17-420c-828f-0e0998d7effb', value='wood'), lock=Lock(name='Lock', id='2a15b9d6-c783-4c52-8c8f-abf913a66e68', locked=Locked(name='Locked', id='3f2b1d1f-0205-4bf7-ad7b-469c2bb1d373', value=True)), gang='gang'),
 DoorLockable(name='Exit', id='535e4f31-86b8-42bc-bb30-9f185c539ec0', locked=Locked(name='Locked', id='73282734-295b-4a06-86e7-a49b7159beb9', value=True), open=Open(name='Open', id='431ec6fd-27f9-447f-a3a3-0ac6d18e60b6', value=True), color=Color(name='Color', id='fc391112-2404-4461-a170-8cf7b19122a6', value='white'), material=Material(name='Materia

In [ ]:
RegistryHolder.all_types()

['Lock', 'Color', 'Open', 'Locked', 'LockableDoor', 'DoorLockable', 'Material']

In [ ]:
def is_open(entity: Door) -> bool:
    if hasattr(entity, 'open') and isinstance(entity.open, Open):
        return entity.open.value
    elif not hasattr(entity, 'open'):
        raise ValueError("The provided entity does not have an 'open' attribute.")
    elif not isinstance(entity.open, Open):
        raise TypeError("The provided entity does not have an 'open' attribute of type Open.")

def is_locked(entity: LockableDoor) -> bool:
    if hasattr(entity, "lock") and hasattr(entity.lock, 'locked') and isinstance(entity.lock.locked, Locked):
        return entity.lock.locked.value
    elif not hasattr(entity, "lock"):
        raise ValueError("The provided entity does not have a 'lock' attribute.")
    elif not hasattr(entity.lock, 'locked'):
        raise ValueError("The provided entity.lock does not have a 'locked' attribute.")
    elif not isinstance(entity.lock.locked, Locked):
        raise TypeError("The provided entity.lock does not have a 'locked' attribute of type Locked.")
    
print(is_open(lockable_door))
print(is_locked(lockable_door))


True
True
